In [1]:
len(range(10,100))

90

In [3]:
from pathlib import Path

In [5]:
Path('foo/bar').relative_to('foo')

WindowsPath('bar')

In [8]:
# find what model is running on a vllm server
import requests
import json
import os
from typing import Dict, Any

def get_vllm_model_info(server_url: str) -> Dict[str, Any]:
	"""
	Get the model information from a VLLM server.

	Args:
		server_url (str): The URL of the VLLM server.

	Returns:
		Dict[str, Any]: A dictionary containing the model information.
	"""
	try:
		response = requests.get(f"{server_url}/v1/models")
		response.raise_for_status()  # Raise an error for bad responses
		model_info = response.json()
		return model_info
	except requests.RequestException as e:
		print(f"Error fetching model info: {e}")
		return {}

info = get_vllm_model_info("http://localhost:8000")
info

{'object': 'list',
 'data': [{'id': 'microsoft/Phi-4-multimodal-instruct',
   'object': 'model',
   'created': 1743704659,
   'owned_by': 'vllm',
   'root': 'microsoft/Phi-4-multimodal-instruct',
   'parent': None,
   'max_model_len': 65536,
   'permission': [{'id': 'modelperm-97ec2978acaa401e8ae91ab95a623042',
     'object': 'model_permission',
     'created': 1743704659,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}]}]}

In [40]:
info['data'][0]['id']

'microsoft/Phi-4-multimodal-instruct'

In [9]:
import openai

In [13]:
openai.ChatCompletion.create(
	  model="microsoft/Phi-4-multimodal-instruct",

  messages=[
		{"role": "user", "content": "Hello!"},
	]
	# temperature=0,
)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [15]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
	api_key=openai_api_key,
	base_url=openai_api_base,
)
completion = client.completions.create(model="microsoft/Phi-4-multimodal-instruct",
									  prompt="Complete the sentence: San Francisco is a")
print("Completion result:", completion)

Completion result: Completion(id='cmpl-9f402c1a31004efdba6df3c46bfb81ec', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' city in _____. Multiple choice:\na. California\nb. New York\n', stop_reason=None, prompt_logprobs=None)], created=1743710032, model='microsoft/Phi-4-multimodal-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=8, total_tokens=24, completion_tokens_details=None, prompt_tokens_details=None))


In [18]:
resp = client.chat.completions.create(model="microsoft/Phi-4-multimodal-instruct",
									  messages=[{'role':'user', 'content': 'Tell me a short joke.'}])
resp

ChatCompletion(id='chatcmpl-db823bf039a64989b0514e7dd29c52e9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Why was the math book sad?\n\nBecause it had too many problems! 😄', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=200020)], created=1743710209, model='microsoft/Phi-4-multimodal-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=9, total_tokens=26, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)

In [21]:
resp.choices[0].completion_tokens

AttributeError: 'Choice' object has no attribute 'completion_tokens'

In [22]:
resp.usage

CompletionUsage(completion_tokens=17, prompt_tokens=9, total_tokens=26, completion_tokens_details=None, prompt_tokens_details=None)

In [41]:
stream = client.chat.completions.create(model="microsoft/Phi-4-multimodal-instruct", stream=True,
										stream_options={"include_usage": True},
									  messages=[{'role':'user', 'content': 'Tell me a short joke.'}])


In [42]:
stream

In [43]:
chunk = next(stream)

In [44]:
chunk.choices[0].delta

ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None)

In [34]:
type(stream)

openai.Stream

In [46]:
chunk

ChatCompletionChunk(id='chatcmpl-7891bb565e254e3a8a4313edcbe77616', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1743716391, model='microsoft/Phi-4-multimodal-instruct', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)

In [47]:
type(chunk)

openai.types.chat.chat_completion_chunk.ChatCompletionChunk

In [48]:
client = openai.OpenAI(api_key='EMPTY', base_url='http://localhost:8000/v1')


In [50]:
str(client.base_url)

'http://localhost:8000/v1/'

In [54]:
client.base_url.host

'localhost'

In [56]:
str(client.base_url)[:-3]

'http://localhost:8000/'

In [57]:
openai.types.chat.chatcompletion.ChatCompletionChunk

AttributeError: module 'openai.types.chat' has no attribute 'chatcompletion'

In [58]:
openai.ChatCompletion.chatcompletion.ChatCompletionChunk

APIRemovedInV1Proxy

In [62]:
openai.ChatCompletion

APIRemovedInV1Proxy

In [5]:
from pathlib import Path
from ludwig.direct_prompt_and_parse.dpp_ttt import DPPforTicTacToe
from ludwig.util.clients import MockEndpoint

In [2]:
dpp = DPPforTicTacToe('bfs', client=MockEndpoint())

In [4]:
dpp.state_desc_template

'{c_sys}\nDescribe the specification of the state representation that "expand" expects such that a person could, given a description of a state, represent it in the desired format to apply the ‘expand’ function.\n{desc_f}'

In [10]:
root = Path().absolute().parent.joinpath('assets/prompts') / 'dpp'
root.mkdir(parents=True, exist_ok=True)

In [16]:
# root.joinpath('respones.txt').write_text(dpp.response_template, encoding='utf-8')

302

In [24]:
class TestContextManager:
	def __enter__(self):
		print("Entering context")
		self.data = {}
		return self.data

	def __exit__(self, exc_type, exc_value, traceback):
		print("Exiting context")
		if exc_type:
			print(f"An exception occurred: {exc_value}")
		self.data['something']= 'foo'

class Top:
	def collect(self):
		return TestContextManager()

cm  = Top()

with cm.collect() as x:
	print(x)
	pass
print(x)

Entering context
{}
Exiting context
{'something': 'foo'}


In [ ]:
with strat.collect_stats() as stats:
	strat.work(something)
print(stats)

In [68]:
# def is_valid_tic_tac_toe(board):
# 	def count(board):
# 		x_count = sum(row.count('X') for row in board)
# 		o_count = sum(row.count('O') for row in board)
# 		return x_count, o_count
#
# 	def win(player):
# 		# Check rows, columns, and diagonals
# 		for i in range(3):
# 			if all(board[i][j] == player for j in range(3)):  # row
# 				return True
# 			if all(board[j][i] == player for j in range(3)):  # column
# 				return True
# 		if all(board[i][i] == player for i in range(3)):      # main diagonal
# 			return True
# 		if all(board[i][2 - i] == player for i in range(3)):  # anti-diagonal
# 			return True
# 		return False
#
# 	x_count, o_count = count(board)
#
# 	# Rule: X always goes first
# 	if o_count > x_count or x_count > o_count + 1:
# 		return False
#
# 	x_win = win('X')
# 	o_win = win('O')
#
# 	# Rule: Both players cannot win simultaneously
# 	if x_win and o_win:
# 		return False
#
# 	# Rule: If X wins, X must have one more move
# 	if x_win and x_count != o_count + 1:
# 		return False
#
# 	# Rule: If O wins, X and O must have the same number of moves
# 	if o_win and x_count != o_count:
# 		return False
#
# 	return True
#
# def check_winner(board):
# 	def win(player):
# 		# Check rows, columns, and diagonals
# 		for i in range(3):
# 			if all(board[i][j] == player for j in range(3)):  # row
# 				return True
# 			if all(board[j][i] == player for j in range(3)):  # column
# 				return True
# 		if all(board[i][i] == player for i in range(3)):      # main diagonal
# 			return True
# 		if all(board[i][2 - i] == player for i in range(3)):  # anti-diagonal
# 			return True
# 		return False
#
# 	if win('X'):
# 		return 'X'
# 	elif win('O'):
# 		return 'O'
# 	else:
# 		return None
#
# from itertools import product
# def generate_ttt_states():
# 	for raw in product(*[['X', 'O', ' '] for _ in range(9)]):
# 		board = [list(raw[i:i+3]) for i in range(0, 9, 3)]
# 		if is_valid_tic_tac_toe(board):
# 			yield board
#
# def generate_next_states(state):
# 	# Count pieces
# 	x_count = sum(row.count('X') for row in state)
# 	o_count = sum(row.count('O') for row in state)
#
# 	# Infer next player
# 	if x_count == o_count:
# 		player = 'X'
# 	elif x_count == o_count + 1:
# 		player = 'O'
# 	else:
# 		raise ValueError("Invalid board state: too many Xs or Os")
#
# 	# Generate next states
# 	next_states = []
# 	for i in range(3):
# 		for j in range(3):
# 			if state[i][j] == ' ':
# 				new_state = [row[:] for row in state]
# 				new_state[i][j] = player
# 				next_states.append(new_state)
# 	return next_states
#
# def state_value(state):
# 	winner = check_winner(state)
# 	if winner == 'X':
# 		return 1
# 	elif winner == 'O':
# 		return -1
#
# 	options = [state_value(state) for state in generate_next_states(state)]
# 	return sum(options) / len(options) if options else 0


In [10]:
states = list(generate_ttt_states())
len(states)

5478

In [22]:
from tqdm.notebook import tqdm

In [23]:
to_code = lambda state: ''.join(''.join(row) for row in state)
exp_values = {to_code(state): state_value(state) for state in tqdm(states)}


  0%|          | 0/5478 [00:00<?, ?it/s]

In [32]:
# import json
# json.dump(exp_values, open('expectimax.json', 'w'), indent=2)

In [26]:
exp_values[' '*9]

0.2968253968253968

In [31]:
exp_values['XXO O    ']

-0.3333333333333333

In [13]:
from collections import Counter

In [14]:
Counter(check_winner(tuple(state)) for state in states)

Counter({None: 4536, 'X': 626, 'O': 316})

In [17]:
sel = [state for state in states if check_winner(tuple(state)) == 'X']
len(sel)

626

In [76]:
def check_winner(state):
	lines = [
		state[0:3], state[3:6], state[6:9],  # rows
		state[0::3], state[1::3], state[2::3],  # columns
		state[0::4], state[2:7:2]  # diagonals
	]
	for line in lines:
		if line == 'XXX':
			return 'X'
		if line == 'OOO':
			return 'O'
	return None

def is_full(state):
	return ' ' not in state

from functools import lru_cache

def generate_next_states(state):
	if check_winner(state) is not None:
		return []
	is_x_turn = state.count('X') == state.count('O')
	next_states = []
	for i in range(9):
		if state[i] == ' ':
			new_state = state[:i] + ('X' if is_x_turn else 'O') + state[i+1:]
			next_states.append(new_state)
	return next_states


@lru_cache(maxsize=None)
def minimax(state):
	winner = check_winner(state)
	if winner == 'X':
		return 1
	if winner == 'O':
		return -1
	if is_full(state):
		return 0
	is_x_turn = state.count('X') == state.count('O')
	return (max if is_x_turn else min)(minimax(option) for option in generate_next_states(state))

@lru_cache(maxsize=None)
def expectimax(state):
	winner = check_winner(state)
	if winner == 'X':
		return 1
	if winner == 'O':
		return -1
	if is_full(state):
		return 0
	is_x_turn = state.count('X') == state.count('O')
	values = [expectimax(option) for option in generate_next_states(state)]
	return sum(values) / len(values) if values else 0


In [78]:
all_states = []
past_states = set()
origin_state = ' ' * 9
todo = [origin_state]
while todo:
	state = todo.pop()
	all_states.append(state)
	past_states.add(state)
	for next_state in generate_next_states(state):
		if next_state not in past_states:
			todo.append(next_state)
len(all_states)

5478

In [79]:
all_states

['         ',
 '        X',
 '       OX',
 '      XOX',
 '     OXOX',
 '    XOXOX',
 '   OXOXOX',
 '  XOXOXOX',
 ' X OXOXOX',
 ' XOOXOXOX',
 'XXOOXOXOX',
 'OX OXOXOX',
 'OXXOXOXOX',
 'X  OXOXOX',
 '  O XOXOX',
 '  OXXOXOX',
 ' OOXXOXOX',
 'XOOXXOXOX',
 'O OXXOXOX',
 'OXOXXOXOX',
 ' XO XOXOX',
 'OXO XOXOX',
 'X O XOXOX',
 ' O  XOXOX',
 ' O XXOXOX',
 'OO XXOXOX',
 'OOXXXOXOX',
 ' OX XOXOX',
 'XO  XOXOX',
 'O   XOXOX',
 'O  XXOXOX',
 'O X XOXOX',
 'OX  XOXOX',
 '   X OXOX',
 '   XOOXOX',
 '  XXOOXOX',
 ' OXXOOXOX',
 'O XXOOXOX',
 'OXXXOOXOX',
 ' X XOOXOX',
 ' XOXOOXOX',
 'XXOXOOXOX',
 'OX XOOXOX',
 'X  XOOXOX',
 '  OX OXOX',
 ' XOX OXOX',
 'OXOX OXOX',
 'X OX OXOX',
 ' O X OXOX',
 ' OXX OXOX',
 'OOXX OXOX',
 'XO X OXOX',
 'O  X OXOX',
 'O XX OXOX',
 'OX X OXOX',
 '  X  OXOX',
 '  X OOXOX',
 ' XX OOXOX',
 ' XXOOOXOX',
 'OXX OOXOX',
 'X X OOXOX',
 'X XOOOXOX',
 'XOX OOXOX',
 '  XO OXOX',
 ' XXO OXOX',
 'OXXO OXOX',
 'X XO OXOX',
 'XOXO OXOX',
 'XOXOXOXOX',
 ' OX  OXOX',
 'XOX  OXOX',
 'O X 

In [83]:
mm_values = {state: minimax(state) for state in tqdm(all_states)}
Counter(mm_values.values()), mm_values[origin_state]

  0%|          | 0/5478 [00:00<?, ?it/s]

(Counter({1: 2936, -1: 1474, 0: 1068}), 0)

In [85]:
ex_values = {state: expectimax(state) for state in tqdm(all_states)}
ex_values[origin_state]

  0%|          | 0/5478 [00:00<?, ?it/s]

0.2968253968253968

In [86]:
mm_values['XXOOXOX  ']

-1

In [87]:
import json
json.dump(mm_values, open('minimax.json', 'w'), indent=2)
json.dump(ex_values, open('expectimax.json', 'w'), indent=2)

In [34]:
mm_values = {state: minimax_str(state, True) for state in tqdm(exp_values)}

  0%|          | 0/5478 [00:00<?, ?it/s]

In [36]:
Counter(mm_values.values())

Counter({1: 4473, 0: 533, -1: 472})

In [37]:
mm_values[' '*9]

0

In [44]:
from tabulate import tabulate

In [47]:
inits = ["OO X  X X", "OX X XO  ", "OXX   O X", "O XX  X O"]
for init in inits:
	print(f'{init!r} - {mm_values[init]:.1f} - {exp_values[init]:.1f}')
	tbl = []
	for state in generate_next_states(init):
		tbl.append((state, state[4] == 'O', mm_values[state], exp_values[state]))
	print(tabulate(tbl, headers=['State', 'Minimax', 'Expectimax'], tablefmt='grid'))
	print()

'OO X  X X' - 1.0 - -0.2
+-----------+---------+-----------+--------------+
|           | State   |   Minimax |   Expectimax |
+===========+=========+===========+==============+
| OOOX  X X | False   |        -1 |    -1        |
+-----------+---------+-----------+--------------+
| OO XO X X | True    |         1 |     0        |
+-----------+---------+-----------+--------------+
| OO X OX X | False   |         1 |     0.666667 |
+-----------+---------+-----------+--------------+
| OO X  XOX | False   |        -1 |    -0.333333 |
+-----------+---------+-----------+--------------+

'OX X XO  ' - 1.0 - 0.2
+-----------+---------+-----------+--------------+
|           | State   |   Minimax |   Expectimax |
+===========+=========+===========+==============+
| OXOX XO   | False   |         1 |     0.333333 |
+-----------+---------+-----------+--------------+
| OX XOXO   | True    |        -1 |    -0.333333 |
+-----------+---------+-----------+--------------+
| OX X XOO  | False   |         

In [48]:
all_states = list(mm_values)

In [54]:
candidates = [state for state in all_states if state.count(' ') == 4 and state[4] == ' ' and check_winner_str(state) is None]
len(candidates)

520

In [55]:
candidates[0]

'XXOX O   '

In [58]:
cand_vals = [{n: mm_values[n] for n in generate_next_states(state)} for state in candidates]
gold = []
for state, vals in zip(candidates, cand_vals):
	m = list(state)
	m[4] = 'O'
	m = ''.join(m)
	if vals[m] < min(v for s, v in vals.items() if s != m):
		gold.append(state)
len(gold)

112

In [59]:
gold # where taking the middle is actually the best

['XXOO   X ',
 'XXOO    X',
 'XXO  O  X',
 'XXO   OX ',
 'XXO   O X',
 'XXO    OX',
 'XX O O X ',
 'XX O O  X',
 'XOXO  X  ',
 'XOX  O  X',
 'XOX   XO ',
 'XOX    OX',
 'XOO  X  X',
 'XO X XO  ',
 'XO X X O ',
 'XO X  O X',
 'XO X   OX',
 'XO   XXO ',
 'XO   X OX',
 'XO    XOX',
 'XO    OXX',
 'X XO OX  ',
 'X XO O X ',
 'X XO O  X',
 'X OX XO  ',
 'X OX  O X',
 'X OO X  X',
 'X O  XOX ',
 'X O  XO X',
 'X O  X OX',
 'X O   OXX',
 'X  X OO X',
 'X  X  OOX',
 'X  O OX X',
 'X  O O XX',
 'X  O  XOX',
 'X  O  OXX',
 'X    OOXX',
 'OXXO  X  ',
 'OXX  OX  ',
 'OXX  O X ',
 'OXX   XO ',
 'OXX   X O',
 'OXX    XO',
 'OXOX X   ',
 'OXOX   X ',
 'OXO  X X ',
 'OX X XO  ',
 'OX X X O ',
 'OX X X  O',
 'OX X O X ',
 'OX X  OX ',
 'OX X   XO',
 'OX   XX O',
 'OX   X XO',
 'OX    XXO',
 'OOXX  X  ',
 'O XX X  O',
 'O XX OX  ',
 'O XX  XO ',
 'O XX  X O',
 'O XX   XO',
 'O X  XX O',
 'O X   XXO',
 'O  X XXO ',
 'O  X XX O',
 'O  X XOX ',
 'O  X X XO',
 ' XXO OX  ',
 ' XXO O X ',
 ' XOX XO  ',
 ' XOX

In [67]:
# print(gold) # ['XXOO   X ', 'XXOO    X', 'XXO  O  X', 'XXO   OX ', 'XXO   O X', 'XXO    OX', 'XX O O X ', 'XX O O  X', 'XOXO  X  ', 'XOX  O  X', 'XOX   XO ', 'XOX    OX', 'XOO  X  X', 'XO X XO  ', 'XO X X O ', 'XO X  O X', 'XO X   OX', 'XO   XXO ', 'XO   X OX', 'XO    XOX', 'XO    OXX', 'X XO OX  ', 'X XO O X ', 'X XO O  X', 'X OX XO  ', 'X OX  O X', 'X OO X  X', 'X O  XOX ', 'X O  XO X', 'X O  X OX', 'X O   OXX', 'X  X OO X', 'X  X  OOX', 'X  O OX X', 'X  O O XX', 'X  O  XOX', 'X  O  OXX', 'X    OOXX', 'OXXO  X  ', 'OXX  OX  ', 'OXX  O X ', 'OXX   XO ', 'OXX   X O', 'OXX    XO', 'OXOX X   ', 'OXOX   X ', 'OXO  X X ', 'OX X XO  ', 'OX X X O ', 'OX X X  O', 'OX X O X ', 'OX X  OX ', 'OX X   XO', 'OX   XX O', 'OX   X XO', 'OX    XXO', 'OOXX  X  ', 'O XX X  O', 'O XX OX  ', 'O XX  XO ', 'O XX  X O', 'O XX   XO', 'O X  XX O', 'O X   XXO', 'O  X XXO ', 'O  X XX O', 'O  X XOX ', 'O  X X XO', ' XXO OX  ', ' XXO O X ', ' XOX XO  ', ' XOX X O ', ' XOX X  O', ' XOX  OX ', ' XOX  O X', ' XOO X X ', ' XO  XOX ', ' XO  X XO', ' XO   OXX', ' X X OOX ', ' X X  OXO', ' X O X XO', ' X O OXX ', ' X O OX X', ' X O O XX', ' X O  XXO', ' X   XOXO', ' X   OOXX', ' OXX X O ', ' OXX X  O', ' OXX  XO ', ' OXX   OX', ' OX  XXO ', ' OX  XX O', ' OX   XXO', ' OX   XOX', ' O X XXO ', ' O X XOX ', ' O X X XO', ' O X X OX', '  XO XX O', '  XO OXX ', '  XO OX X', '  XO  XXO', '  X  XXOO', '  X  OXXO', '  X  OXOX', '  OX XOX ', '  OX XO X', '  OX X XO', '  OX X OX', '   X XOXO']

# print(candidates) # ['XXOX O   ', 'XXOX  O  ', 'XXOX   O ', 'XXOX    O', 'XXOO X   ', 'XXOO  X  ', 'XXOO   X ', 'XXOO    X', 'XXO  XO  ', 'XXO  X O ', 'XXO  X  O', 'XXO  OX  ', 'XXO  O X ', 'XXO  O  X', 'XXO   XO ', 'XXO   X O', 'XXO   OX ', 'XXO   O X', 'XXO    XO', 'XXO    OX', 'XX X OO  ', 'XX X O O ', 'XX X O  O', 'XX X  OO ', 'XX X  O O', 'XX X   OO', 'XX O XO  ', 'XX O X O ', 'XX O X  O', 'XX O OX  ', 'XX O O X ', 'XX O O  X', 'XX O  XO ', 'XX O  X O', 'XX O  OX ', 'XX O  O X', 'XX O   XO', 'XX O   OX', 'XX   XOO ', 'XX   XO O', 'XX   X OO', 'XX   OXO ', 'XX   OX O', 'XX   OOX ', 'XX   OO X', 'XX   O XO', 'XX   O OX', 'XX    XOO', 'XX    OXO', 'XX    OOX', 'XOXX O   ', 'XOXX  O  ', 'XOXX   O ', 'XOXX    O', 'XOXO X   ', 'XOXO  X  ', 'XOXO   X ', 'XOXO    X', 'XOX  XO  ', 'XOX  X O ', 'XOX  X  O', 'XOX  OX  ', 'XOX  O X ', 'XOX  O  X', 'XOX   XO ', 'XOX   X O', 'XOX   OX ', 'XOX   O X', 'XOX    XO', 'XOX    OX', 'XOOX X   ', 'XOOX   X ', 'XOOX    X', 'XOO  XX  ', 'XOO  X X ', 'XOO  X  X', 'XOO   XX ', 'XOO   X X', 'XOO    XX', 'XO X XO  ', 'XO X X O ', 'XO X X  O', 'XO X O X ', 'XO X O  X', 'XO X  OX ', 'XO X  O X', 'XO X   XO', 'XO X   OX', 'XO O XX  ', 'XO O X X ', 'XO O X  X', 'XO O  XX ', 'XO O  X X', 'XO O   XX', 'XO   XXO ', 'XO   XX O', 'XO   XOX ', 'XO   XO X', 'XO   X XO', 'XO   X OX', 'XO   OXX ', 'XO   OX X', 'XO   O XX', 'XO    XXO', 'XO    XOX', 'XO    OXX', 'X XX OO  ', 'X XX O O ', 'X XX O  O', 'X XX  OO ', 'X XX  O O', 'X XX   OO', 'X XO XO  ', 'X XO X O ', 'X XO X  O', 'X XO OX  ', 'X XO O X ', 'X XO O  X', 'X XO  XO ', 'X XO  X O', 'X XO  OX ', 'X XO  O X', 'X XO   XO', 'X XO   OX', 'X X  XOO ', 'X X  XO O', 'X X  X OO', 'X X  OXO ', 'X X  OX O', 'X X  OOX ', 'X X  OO X', 'X X  O XO', 'X X  O OX', 'X X   XOO', 'X X   OXO', 'X X   OOX', 'X OX XO  ', 'X OX X O ', 'X OX X  O', 'X OX O X ', 'X OX O  X', 'X OX  OX ', 'X OX  O X', 'X OX   XO', 'X OX   OX', 'X OO XX  ', 'X OO X X ', 'X OO X  X', 'X OO  XX ', 'X OO  X X', 'X OO   XX', 'X O  XXO ', 'X O  XX O', 'X O  XOX ', 'X O  XO X', 'X O  X XO', 'X O  X OX', 'X O  OXX ', 'X O  OX X', 'X O  O XX', 'X O   XXO', 'X O   XOX', 'X O   OXX', 'X  X XOO ', 'X  X XO O', 'X  X X OO', 'X  X OOX ', 'X  X OO X', 'X  X O XO', 'X  X O OX', 'X  X  OXO', 'X  X  OOX', 'X  O XXO ', 'X  O XX O', 'X  O XOX ', 'X  O XO X', 'X  O X XO', 'X  O X OX', 'X  O OXX ', 'X  O OX X', 'X  O O XX', 'X  O  XXO', 'X  O  XOX', 'X  O  OXX', 'X    XXOO', 'X    XOXO', 'X    XOOX', 'X    OXXO', 'X    OXOX', 'X    OOXX', 'OXXX O   ', 'OXXX  O  ', 'OXXX   O ', 'OXXX    O', 'OXXO X   ', 'OXXO  X  ', 'OXXO   X ', 'OXXO    X', 'OXX  XO  ', 'OXX  X O ', 'OXX  X  O', 'OXX  OX  ', 'OXX  O X ', 'OXX  O  X', 'OXX   XO ', 'OXX   X O', 'OXX   OX ', 'OXX   O X', 'OXX    XO', 'OXX    OX', 'OXOX X   ', 'OXOX  X  ', 'OXOX   X ', 'OXOX    X', 'OXO  XX  ', 'OXO  X X ', 'OXO  X  X', 'OXO   XX ', 'OXO   X X', 'OXO    XX', 'OX X XO  ', 'OX X X O ', 'OX X X  O', 'OX X OX  ', 'OX X O X ', 'OX X O  X', 'OX X  XO ', 'OX X  X O', 'OX X  OX ', 'OX X  O X', 'OX X   XO', 'OX X   OX', 'OX O XX  ', 'OX O X X ', 'OX O X  X', 'OX O  XX ', 'OX O  X X', 'OX O   XX', 'OX   XXO ', 'OX   XX O', 'OX   XOX ', 'OX   XO X', 'OX   X XO', 'OX   X OX', 'OX   OXX ', 'OX   OX X', 'OX   O XX', 'OX    XXO', 'OX    XOX', 'OX    OXX', 'OOXX X   ', 'OOXX  X  ', 'OOXX   X ', 'OOXX    X', 'OOX  XX  ', 'OOX  X X ', 'OOX   XX ', 'OOX   X X', 'OOX    XX', 'OO X XX  ', 'OO X X X ', 'OO X X  X', 'OO X  XX ', 'OO X  X X', 'OO X   XX', 'OO   XXX ', 'OO   XX X', 'OO   X XX', 'O XX XO  ', 'O XX X O ', 'O XX X  O', 'O XX OX  ', 'O XX O X ', 'O XX O  X', 'O XX  XO ', 'O XX  X O', 'O XX  OX ', 'O XX  O X', 'O XX   XO', 'O XX   OX', 'O XO XX  ', 'O XO X X ', 'O XO  XX ', 'O XO  X X', 'O XO   XX', 'O X  XXO ', 'O X  XX O', 'O X  XOX ', 'O X  X XO', 'O X  OXX ', 'O X  OX X', 'O X  O XX', 'O X   XXO', 'O X   XOX', 'O X   OXX', 'O OX XX  ', 'O OX X X ', 'O OX X  X', 'O OX  XX ', 'O OX  X X', 'O OX   XX', 'O O  XXX ', 'O O  XX X', 'O O  X XX', 'O  X XXO ', 'O  X XX O', 'O  X XOX ', 'O  X XO X', 'O  X X XO', 'O  X X OX', 'O  X OXX ', 'O  X OX X', 'O  X O XX', 'O  X  XXO', 'O  X  XOX', 'O  X  OXX', 'O  O XXX ', 'O  O XX X', 'O  O X XX', 'O    XXXO', 'O    XXOX', 'O    XOXX', ' XXX OO  ', ' XXX O O ', ' XXX O  O', ' XXX  OO ', ' XXX  O O', ' XXX   OO', ' XXO XO  ', ' XXO X O ', ' XXO X  O', ' XXO OX  ', ' XXO O X ', ' XXO O  X', ' XXO  XO ', ' XXO  X O', ' XXO  OX ', ' XXO  O X', ' XXO   XO', ' XXO   OX', ' XX  XOO ', ' XX  XO O', ' XX  X OO', ' XX  OXO ', ' XX  OX O', ' XX  OOX ', ' XX  OO X', ' XX  O XO', ' XX  O OX', ' XX   XOO', ' XX   OXO', ' XX   OOX', ' XOX XO  ', ' XOX X O ', ' XOX X  O', ' XOX OX  ', ' XOX O X ', ' XOX O  X', ' XOX  XO ', ' XOX  X O', ' XOX  OX ', ' XOX  O X', ' XOX   XO', ' XOX   OX', ' XOO XX  ', ' XOO X X ', ' XOO X  X', ' XOO  XX ', ' XOO  X X', ' XOO   XX', ' XO  XXO ', ' XO  XX O', ' XO  XOX ', ' XO  XO X', ' XO  X XO', ' XO  X OX', ' XO  OXX ', ' XO  OX X', ' XO  O XX', ' XO   XXO', ' XO   XOX', ' XO   OXX', ' X X XOO ', ' X X XO O', ' X X X OO', ' X X OXO ', ' X X OX O', ' X X OOX ', ' X X OO X', ' X X O XO', ' X X O OX', ' X X  XOO', ' X X  OXO', ' X X  OOX', ' X O XXO ', ' X O XX O', ' X O XOX ', ' X O XO X', ' X O X XO', ' X O X OX', ' X O OXX ', ' X O OX X', ' X O O XX', ' X O  XXO', ' X O  XOX', ' X O  OXX', ' X   XXOO', ' X   XOXO', ' X   XOOX', ' X   OXXO', ' X   OXOX', ' X   OOXX', ' OXX XO  ', ' OXX X O ', ' OXX X  O', ' OXX OX  ', ' OXX O X ', ' OXX O  X', ' OXX  XO ', ' OXX  X O', ' OXX  OX ', ' OXX  O X', ' OXX   XO', ' OXX   OX', ' OXO XX  ', ' OXO X X ', ' OXO  XX ', ' OXO  X X', ' OXO   XX', ' OX  XXO ', ' OX  XX O', ' OX  XOX ', ' OX  X XO', ' OX  OXX ', ' OX  OX X', ' OX  O XX', ' OX   XXO', ' OX   XOX', ' OX   OXX', ' OOX XX  ', ' OOX X X ', ' OOX X  X', ' OOX  XX ', ' OOX  X X', ' OOX   XX', ' OO  XXX ', ' OO  XX X', ' OO  X XX', ' O X XXO ', ' O X XX O', ' O X XOX ', ' O X XO X', ' O X X XO', ' O X X OX', ' O X OXX ', ' O X OX X', ' O X O XX', ' O X  XXO', ' O X  XOX', ' O X  OXX', ' O O XXX ', ' O O XX X', ' O O X XX', ' O   XXXO', ' O   XXOX', ' O   XOXX', '  XX XOO ', '  XX XO O', '  XX X OO', '  XX OXO ', '  XX OX O', '  XX OOX ', '  XX OO X', '  XX O XO', '  XX O OX', '  XX  XOO', '  XX  OXO', '  XX  OOX', '  XO XXO ', '  XO XX O', '  XO XOX ', '  XO X XO', '  XO OXX ', '  XO OX X', '  XO O XX', '  XO  XXO', '  XO  XOX', '  XO  OXX', '  X  XXOO', '  X  XOXO', '  X  OXXO', '  X  OXOX', '  X  OOXX', '  OX XXO ', '  OX XX O', '  OX XOX ', '  OX XO X', '  OX X XO', '  OX X OX', '  OX OXX ', '  OX OX X', '  OX O XX', '  OX  XXO', '  OX  XOX', '  OX  OXX', '  OO XXX ', '  OO XX X', '  OO X XX', '  O  XXXO', '  O  XXOX', '  O  XOXX', '   X XXOO', '   X XOXO', '   X XOOX', '   X OXXO', '   X OXOX', '   X OOXX', '   O XXXO', '   O XXOX', '   O XOXX']

In [1]:
import json
from omnibelt import pformat

In [2]:
products = {'p1': {'title': 'Product 1', 'description': 'Description of product 1'},
			'p2': {'title': 'Product 2', 'description': 'Description of product 2'},
			'p3': {'title': 'Product 3', 'description': 'Description of product 3'}}
selected1 = '["p1", "p2"]'

In [8]:
pformat('{[x for x in range(3)]}')

KeyError: "Variable 'x' not found in provided args or kwargs"

In [16]:


# tmpl = "The user selected the following recommendation:\n{'- '+'\n- '.join([f'**{products[pid]['title']}**: {products[pid]['description']}' for pid in json.loads(selected1)])}"
#
# print(pformat(tmpl, products=products, json=json, selected1=selected1))


In [6]:
# eval('''[f'**{products[pid]["title"]}**: {products[pid]["description"]}' for pid in json.loads(selected1)]''')

['**Product 1**: Description of product 1',
 '**Product 2**: Description of product 2']

In [15]:

tmpl = "The user selected the following recommendation:\n{'- '+'\n- '.join([f'**{products[pid]['title']}**: {products[pid]['description']}' for pid in json.loads(selected1)])}"

print(pformat_new(tmpl, products=products, json=json, selected1=selected1))

KeyError: "'- '+'\n- '"

In [14]:
pformat_new('{[x for x in range(3)]}')

IndexError: Replacement index 0 out of range for positional args tuple